In [1]:
import Bio

In [2]:
dna = 'TGGGCCTCATATTTATCCTATATACCATGTTCGTATGGTGGCGCGATGTTCTACGTGAATCCACGTTCGAAGGACATCATACCAAAGTCGTACAATTAGGACCTCGATATGGTTTTATTCTGTTTATCGTATCGGAGGTTATGTTCTTTTTTGCTCTTTTTCGGGCTTCTTCTCATTCTTCTTTGGCACCTACGGTAGAG'
from Bio.Blast import NCBIWWW

In [3]:
result_handle = NCBIWWW.qblast("blastn", "nt", dna)

In [4]:
from Bio.Blast import NCBIXML
blast_record = NCBIXML.read(result_handle)

In [5]:
blast_record

In [6]:
len(blast_record.alignments)

50

In [7]:
E_VALUE_THRESH = 0.01
for alignment in blast_record.alignments:
    for hsp in alignment.hsps:
        if hsp.expect < E_VALUE_THRESH:
            print('***Alignment***')
            print('sequence:', alignment.title)
            print('length:', alignment.length)
            print('e value:', hsp.expect)
            print(hsp.query)
            print(hsp.match)
            print(hsp.sbjct)

***Alignment***
sequence: gi|1783584753|gb|MN651324.1| Nicotiana tabacum strain zhongyan90 cytoplasmic male sterility(CMS) line cultivar MSzhongyan90 mitochondrion, complete genome
length: 530869
e value: 6.76856e-96
TGGGCCTCATATTTATCCTATATACCATGTTCGTATGGTGGCGCGATGTTCTACGTGAATCCACGTTCGAAGGACATCATACCAAAGTCGTACAATTAGGACCTCGATATGGTTTTATTCTGTTTATCGTATCGGAGGTTATGTTCTTTTTTGCTCTTTTTCGGGCTTCTTCTCATTCTTCTTTGGCACCTACGGTAGAG
||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
TGGGCCTCATATTTATCCTATATACCATGTTCGTATGGTGGCGCGATGTTCTACGTGAATCCACGTTCGAAGGACATCATACCAAAGTCGTACAATTAGGACCTCGATATGGTTTTATTCTGTTTATCGTATCGGAGGTTATGTTCTTTTTTGCTCTTTTTCGGGCTTCTTCTCATTCTTCTTTGGCACCTACGGTAGAG
***Alignment***
sequence: gi|1783584659|gb|MN651323.1| Nicotiana tabacum strain zhongyan90 maintainer line cultivar zhongyan90 mitochondrion, complete genome
length: 472218
e value

In [9]:
dna = "TGGGCCTCATATTTATCCTATATACCATGTTCGTATGGTGGCGCGATGTTCTACGTGAATCCACGTTCGAAGGACATCATACCAAAGTCGTACAATTAGGACCTCGATATGGTTTTATTCTGTTTATCGTATCGGAGGTTATGTTCTTTTTTGCTCTTTTTCGGGCTTCTTCTCATTCTTCTTTGGCACCTACGGTAGAG"

1. How many records are in the multi-FASTA file?

In [10]:
dna = open("dna2.fasta") 

In [11]:
dna

<_io.TextIOWrapper name='dna2.fasta' mode='r' encoding='UTF-8'>

In [12]:
n = 0
dna.seek(0)
for line in dna:
    if line[0] == '>':
        n += 1
print(n)

18


2. What are the lengths of the sequences in the file? What is the longest sequence and what is the shortest sequence?

In [13]:
dna.seek(0)
seqs = {}
for line in dna:
    line = line.rstrip()
    if line[0] == '>':
        words = line.split()
        name = words[0][1:]
        seqs[name] = ''
    else:
        seqs[name] = seqs[name] + line
        
lengths = [len(seqs[i]) for i in seqs]
print(min(lengths), max(lengths))

115 4894


3. What is the length of the longest ORF appearing in reading frame 2 of any of the sequences?

In [14]:
print(seqs)

{'gi|142022655|gb|EQ086233.1|91': 'CTCGCGTTGCAGGCCGGCGTGTCGCGCAACGACGTGTGGGGCCTGACGGGCAGGGAGGATCTCGGCGGCGCCAACTATGCGGTCTTTCGGCTCGAAAGCCAGTTCCAGACCTCCGACGGCGCGCTGACCGTGCCCGGCTCCGCATTCAGTTCGCAAGCCTACGTCGGGCTCGGCGGCGACTGGGGGACCGTGACGCTCGGGCGCCAGTTCGATTTCGTCGGCGATCTGATGCCGGCTTTCGCGATCGGCGCGAACACGCCGGCCGGCCTGCTCGCGTGGGGCTTGCCGGCGAATGCGTCGGCGGGCGGTGCGCTCGACAACCGCGTGTGGGGCGTCCAGGTGAACAATGCGGTGAAGTACGTGAGCCCGACGTTCGGCGGATTGTCGTTCGGCGGCCTGTGGGGCTTCGGCAACGTGCCCGGCACGGTCGCGCGCAGCAGCGTGCAAAGCGCGATGCTGTCCTACACGCAAGGCGCGTTCAGCGCCGCGCTCGCTTATTTCGGCCAGCACGATGTAACTGCCGGTGGCAATCTGCGCAATTTCTCGGGCGGTGCAGGCTACAACGTCGGGCAGTTCCGCGTCTTCGGCATGGTGTCGGACGTGCGGATCAGCGCCGCCGCGCCGCTGCGGGCCACGACCTATGACGGCGGCTTGACCTATGCGGTCACGCCGGCGTTGCAGCTCGGCGGCGGCTTCCAGTACCAGCAGCGCGGCGGCGACATCGGCTCGGCCAACCAGGTCACGTTGAGCGCCGACTATTCGCTGTCGAAGCGTACCGGCCTTTACGTGGTATTCGCACGCGGGCACGACAGTGCGTATGGCGCGCAGGTCGAGGCGGCGCTCGGCGGGGCGGCGTCCGGCTCGACGCAGACCGCGGTCCGGCTCGGGCTGCGGCATCAGTTCTGACGATGCGCGAGAAACACGGGCTGCCGCGTACGCCGCGCGCGAGCCCGTGTTTTTCCGCC

In [27]:
from Bio import SeqIO
def orf(sequence, rf):
    seq = sequence[rf - 1:]
    max_len_orf = 0
    orf_start = 0

    for i in range(0, len(seq) - 6, 3):
        if seq[i:i + 3] == 'ATG':
            for j in range(i + 3, len(seq) - 3, 3):
                if seq[j:j + 3] in ['TAA', 'TAG', 'TGA']:
                    len_orf = j+3-i
                    if len_orf > max_len_orf:
                        max_len_orf = len_orf
                        orf_start = i
                    break

    return max_len_orf, orf_start


In [28]:
lengths2 = []
for i in seqs:
    lengths2.append(orf(seqs[i], 2))
print(lengths2)
print(max(lengths2))

[(237, 816), (0, 0), (1185, 429), (528, 156), (1281, 1515), (552, 162), (156, 273), (138, 180), (33, 693), (420, 1572), (504, 255), (822, 756), (1458, 3069), (27, 201), (126, 60), (279, 678), (0, 0), (570, 1539)]
(1458, 3069)


5. What is the starting position of the longest ORF in reading frame 3 in any of the sequences? The position should indicate the character number where the ORF begins. For instance, the following ORF:

In [32]:
lengths3 = []
for i in seqs:
    lengths3.append(orf(seqs[i], 3))

print(max(lengths3))

(1821, 633)


6. What is the length of the longest ORF appearing in any sequence and in any forward reading frame?

In [34]:
lengths = []
for i in seqs:
    lengths.append(orf(seqs[i], 1))
    lengths.append(orf(seqs[i], 2))
    lengths.append(orf(seqs[i], 3))
    
print(max(lengths))

(2394, 384)


7. What is the length of the longest forward ORF that appears in the sequence with the identifier  gi|142022655|gb|EQ086233.1|16?

In [35]:
lengths = []
for i in seqs:
    if i == 'gi|142022655|gb|EQ086233.1|16':
        lengths.append(orf(seqs[i], 1))
        lengths.append(orf(seqs[i], 2))
        lengths.append(orf(seqs[i], 3))

print(max(lengths))

(1644, 1437)


8. Find the most frequently occurring repeat of length 6 in all sequences. How many times does it occur in all?

In [53]:
def find_repeats(sequence, repeat_num):
    repeats = []
    for i in range(len(sequence)):
        repeats.append(sequence[i : i + repeat_num])
    return repeats

In [54]:
repeats6 = []
for i in seqs:
    r = find_repeats(seqs[i], 6)
    for j in r:
        repeats6.append(j)

In [55]:
print(max(set(repeats6),key=repeats6.count))

GCGCGC


In [56]:
print(repeats6.count(max(set(repeats6), key=repeats6.count)))

153


In [57]:
from collections import Counter
print("Counter of repeats of length n, most common 5: ", Counter(repeats6).most_common(5))


Counter of repeats of length n, most common 5:  [('GCGCGC', 153), ('CGCGCG', 151), ('GCCGCG', 147), ('GCGCCG', 135), ('CGCGGC', 131)]


In [58]:
repeats12 = []
for i in seqs:
    r = find_repeats(seqs[i], 12)
    for j in r:
        repeats12.append(j)
print("Counter of repeats of length 12, most common 5: ", Counter(repeats12).most_common(5))


Counter of repeats of length 12, most common 5:  [('CATTCGCCATTC', 10), ('ATTCGCCATTCG', 10), ('TTCGCCATTCGC', 10), ('TCGCCATTCGCC', 10), ('CGCCATTCGCCA', 9)]


In [60]:
repeats7 = []
for i in seqs:
    r = find_repeats(seqs[i], 7)
    for j in r:
        repeats7.append(j)
print("Counter of repeats of length 7, most common 5: ", Counter(repeats7).most_common(5))

Counter of repeats of length 7, most common 5:  [('CGCGCCG', 63), ('CGCCGCG', 62), ('GCCGCGC', 61), ('GCGCGCG', 59), ('GCGCGGC', 58)]
